<a href="https://colab.research.google.com/github/wlsisl/ESAA_22-2/blob/main/%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%ED%9A%8C%EA%B7%801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 단순 선형 회귀 분석**
- 전복의 나이를 예측하는 선형회귀모델을 생성하세요.
- 전복의 ‘성별’, ‘키’, ‘지름’, ‘높이’, ‘전체무게’, ‘몸통무게’, ‘내장무게’, ‘껍질무게’를 이용해 ‘껍질의 고리 수’를 예측한 뒤, **예측된 ‘껍질의 고리 수’에 1.5를 더하면 전복의 나이**가 됩니다.

In [1]:
# 기본 모듈 불러오기
import numpy as np
import pandas as pd

**1) 데이터 load 및 변형** 

In [4]:
# 데이터 로드
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/ESAA_YB/abalone.csv')
data.head()
print(data.shape)

# 성별 M은 Male, F는 Female, I는 Infant 이므로 따로 열 만들기
for label in "MFI":
    data[label] = data["Sex"] == label
data.drop('Sex', axis=1, inplace=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(4177, 9)


**2) X, y 선택**
: y는 Rings열, X는 Rings열을 제외한 나머지를 선택하되 전부 실수가 되도록 한다.

In [29]:
# X,y 데이터 선택
y=data.Rings.values
data.drop('Rings',axis=1,inplace=True)

X = data.values.astype(float)

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,M,F,I
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,True,False,False
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,True,False,False
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,False,True,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,True,False,False
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,False,False,True
...,...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,False,True,False
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,True,False,False
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,True,False,False
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,False,True,False


 **3) train/test set 분리**

In [12]:
# 필요한 모듈 불러오기
from sklearn.model_selection import train_test_split

In [32]:
# train과 test set 분리 (train:test = 7:3 비율로)

X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=100)

**4) 선형회귀모델 생성, 모델 예측치 구하기**

In [14]:
#필요한 모듈 불러오기
from sklearn.linear_model import LinearRegression


In [41]:
#선형회귀모델 생성 및 훈련
lr = LinearRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_test)

In [49]:
# 모델 예측치 구하기
y_preds
# 모델 예측치를 활용해 최종적으로 전복의 나이를 예측
y_preds+1.5

array([12.984375 , 14.703125 , 12.4375   , ..., 17.8125   , 12.9609375,
        9.8359375])

**5) 모델 평가: MSE, RMSE, R2 score, corr 구하기**

In [19]:
#필요한 모듈 불러오기
from sklearn.metrics import mean_squared_error, r2_score

- MSE, RMSE 

In [45]:
#mse, rmse
mse= mean_squared_error(y_test,y_preds)
rmse = np.sqrt(mse)

print('Mse:{0:.3f}, RMSEE:{1:.3F}'.format(mse, rmse))


Mse:4.931, RMSEE:2.221


- R2 score

In [46]:
#R2 score 측정
R2= r2_score(y_test, y_preds)
R2

0.5336269894594383

- 회귀 절편값

In [47]:
#회귀 절편 값
lr.intercept_

48785761923294.74

- 회귀 계수 값

In [48]:
#회귀 계수 값
np.round(lr.coef_,1)

array([ 1.60000000e+00,  9.20000000e+00,  7.10000000e+00,  1.00000000e+01,
       -2.15000000e+01, -1.04000000e+01,  7.90000000e+00, -4.87857619e+13,
       -4.87857619e+13, -4.87857619e+13])

- 상관계수

Hint: corr 함수 이용.

In [50]:
df = pd.DataFrame({'y_pred' : y_preds, 'y_true' : y_test})
cor = df['y_pred'].corr(df['y_true'])
cor

0.7305459432088361

# **2. Polynomial features**

In [25]:
# PolynomialFeatures 라이브러리 호출
from sklearn.preprocessing import PolynomialFeatures

In [26]:
# 임의 데이터 생성

X = np.arange(6).reshape(3, 2)

df =  pd.DataFrame(X)
df.columns = ['x_1','x_2']
df

,x_1,x_2
0,0,1
1,2,3
2,4,5


In [27]:
# 차원은 2로 설정
poly_features = PolynomialFeatures(degree=2) 
# fit_transform 메소드를 통해 데이터 변환
df_poly = poly_features.fit_transform(df) 
# PolynomialFeatures로 변환 된 데이터를 데이터 프레임 형태로 변환
df_poly = pd.DataFrame(df_poly)

In [28]:
# df_poly의 컬럼을 1,x1,x2,x1^2,x1*x2,x2^2 로 변경
df_poly.columns = ['1','x1','x2','x1^2','x1*x2','x2^2'] 